# Aux function

In [ ]:
# Get audio length
import wave
import time
def get_audio_length(file_path):
    with wave.open(file_path, 'rb') as audio_file:
        # Get the number of frames and the frame rate
        num_frames = audio_file.getnframes()
        frame_rate = audio_file.getframerate()

        # Calculate the duration in seconds
        duration = num_frames / frame_rate

    return duration

# Obtain complete datasets

In [ ]:
# Download training data
!wget https://www.openslr.org/resources/60/train-clean-360.tar.gz

# Extract files
!tar -xf train-clean-360.tar.gz

import os
os.remove('train-clean-360.tar.gz')

# Download training data
!wget https://www.openslr.org/resources/60/train-clean-100.tar.gz

# Extract files
!tar -xf train-clean-100.tar.gz
os.remove('train-clean-100.tar.gz')

--2024-04-25 15:05:02--  https://www.openslr.org/resources/60/train-clean-360.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/60/train-clean-360.tar.gz [following]
--2024-04-25 15:05:02--  https://openslr.elda.org/resources/60/train-clean-360.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27504073644 (26G) [application/x-gzip]
Saving to: ‘train-clean-360.tar.gz’

train-clean-360.tar 100%[===================>]  25.61G  30.6MB/s    in 15m 33s 

2024-04-25 15:20:36 (28.1 MB/s) - ‘train-clean-360.tar.gz’ saved [27504073644/27504073644]

--2024-04-25 15:32:00--  https://www.openslr.org/resources/60/train-clean-100.ta

In [ ]:
import os
import pandas as pd
import glob

def make_speaker_df(dataset):

  speaker_ids = os.listdir(dataset)
  mins_total = []
  mins_14 = []
  mins_10 = []
  n_samples_total = []
  n_samples_14 = []
  n_samples_10 = []

  for speaker_id in speaker_ids:

    # List all wav files in the speaker directory
    wav_files = glob.glob(os.path.join(dataset, speaker_id + '/**/*.wav'), recursive=True)

    # Duración de los archivos del speaker_id
    duration = 0
    duration_14 = 0
    duration_10 = 0
    n_14 = 0
    n_10 = 0
    for archivo in wav_files:

        archivo_seconds = get_audio_length(archivo)

        duration = duration + archivo_seconds

        if archivo_seconds < 14:
          n_14 = n_14 + 1
          duration_14 = duration_14 + archivo_seconds

          if archivo_seconds < 10:
            n_10 = n_10 + 1
            duration_10 = duration_10 + archivo_seconds

    mins_total.append(duration / 60)
    mins_14.append(duration_14 / 60)
    mins_10.append(duration_10 / 60)
    n_samples_total.append(len(wav_files))
    n_samples_14.append(n_14)
    n_samples_10.append(n_10)

  speaker_df = pd.DataFrame({'Dataset': dataset,
                            'Speaker ID': speaker_ids,
                            'Mins total': mins_total,
                            'Mins < 14': mins_14,
                            'Mins < 10': mins_10,
                             'Num samples total': n_samples_total,
                             'Num samples < 14': n_samples_14,
                             'Num samples < 10': n_samples_10})

  return speaker_df


In [ ]:
speaker_df1 = make_speaker_df('LibriTTS/train-clean-360')
speaker_df2 = make_speaker_df('LibriTTS/train-clean-100')


# Select voices to clone and place the audios in the required format

Place your dataset like dataset_raw/{speaker_id}/**/{wav_file}.{any_format} (subfolders and non-ASCII filenames are acceptable)

In [ ]:
speaker_df = pd.concat([speaker_df1, speaker_df2]).sort_values(by='Mins < 10', ascending=False).reset_index(drop=True)

speaker_df = speaker_df.sort_values(by='Mins < 10', ascending=False).reset_index(drop=True)
speaker_df.head(6)

,Dataset,Speaker ID,Mins total,Mins < 14,Mins < 10,Num samples total,Num samples < 14,Num samples < 10
0,LibriTTS/train-clean-360,3922,32.526731,29.699231,26.232228,515,505,487
1,LibriTTS/train-clean-360,8699,30.060960,27.826460,25.896460,487,480,470
2,LibriTTS/train-clean-360,3638,30.889009,27.968004,24.112003,398,388,368
3,LibriTTS/train-clean-360,3307,34.902197,30.299863,23.918853,312,295,262
4,LibriTTS/train-clean-100,8088,28.172064,27.067065,23.082733,378,374,353
5,LibriTTS/train-clean-100,6209,32.299192,27.821192,22.727522,458,443,416


In [ ]:
from IPython.display import Audio, display
import random

for i in range(6):
  dataset = speaker_df.loc[i, 'Dataset']
  speaker_id = speaker_df.loc[i, 'Speaker ID']

  wav_files = glob.glob(os.path.join(dataset, speaker_id + '/**/*.wav'), recursive=True)

  print(dataset)
  print(speaker_id)
  display(Audio(random.choice(wav_files)))

  # 3992: mujer
  # 8699: mujer
  # 3638: mujer
  # 3307: hombre
  # 8088: hombre
  # 6209: hombre

LibriTTS/train-clean-360
3922


LibriTTS/train-clean-360
8699


LibriTTS/train-clean-360
3638


LibriTTS/train-clean-360
3307


LibriTTS/train-clean-100
8088


LibriTTS/train-clean-100
6209


In [ ]:
import shutil
import csv
# shutil.rmtree('OriginalAudiosEnglish')

for i in range(6):
  dataset = speaker_df.loc[i, 'Dataset']
  speaker_id = speaker_df.loc[i, 'Speaker ID']

  wav_files = glob.glob(os.path.join(dataset, speaker_id + '/**/*.wav'), recursive=True)
  new_path = os.path.join('OriginalAudiosEnglish', speaker_id)
  os.makedirs(new_path)

  metadata_file_name = 'metadata_' + str(speaker_id) + '.txt'
  file_path = os.path.join('OriginalAudiosEnglish', metadata_file_name)
  with open(file_path, 'w') as f:
    pass

  for wav_file in wav_files:
    if get_audio_length(wav_file) < 10:
      shutil.copy(wav_file, new_path)

      directory = os.path.dirname(wav_file)
      audio_id = os.path.splitext(os.path.basename(wav_file))[0]
      text_path = os.path.join(directory, audio_id) + '.original.txt'
      original_text = pd.read_csv(text_path, header=None, quoting=csv.QUOTE_NONE, delimiter='\t').iloc[0,0]

      with open(file_path, 'a') as f:
        f.write(audio_id + '|' + original_text + '\n')



# Download

In [ ]:
# # Zip folder
# folder_path = 'OriginalAudiosEnglish'
# shutil.make_archive(folder_path, 'zip', folder_path)

# # Download the zip files
# from google.colab import files
# files.download('OriginalAudiosEnglish.zip')